In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForMaskedLM,XLMRobertaForSequenceClassification

In [5]:
model_name = 'xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base",num_labels=5)
# Define device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import pandas as pd
train_url = '/kaggle/input/rev-corr/train8020.csv'
test_url = '/kaggle/input/rev-corr/test8020.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('/kaggle/input/bangla-stopwords/stopwords_bangla.xlsx',index_col=False)

In [7]:
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [8]:
import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
df_train['Comment'] = df_train['Comment'].apply(lambda x: preprocess(x))
df_test['Comment'] = df_test['Comment'].apply(lambda x:preprocess(x))

In [9]:
import numpy as np
allcats = set(df_train['Category'].dropna().tolist())
allcats.add('Correct')
labeldict = {}
counter = 0
for i in allcats:
    labeldict[i] = counter
    counter += 1
labeldict

{'Multiple Errors': 0,
 'Code Switching': 1,
 'Correct': 2,
 'Grammatical': 3,
 'Spelling': 4}

In [10]:
def manage(x):
    if x in labeldict:
        return labeldict[x]
    else:
        return labeldict['Correct']
df_train['Category'] = df_train['Category'].apply(lambda x:manage(x))
df_test['Category'] = df_test['Category'].apply(lambda x:manage(x))

In [11]:
data_no = 5

# Prepare the training data
train_texts = df_train['Comment'].tolist()
train_labels = df_train['Category'].tolist()

test_texts = df_test['Comment'].tolist()
test_labels = df_test['Category'].tolist()

In [12]:
# Tokenize and encode the training texts
train_encodings = tokenizer(train_texts, truncation=True, max_length=128,
        padding='max_length', return_tensors = 'pt')

# Convert the labels to tensors
train_labels = torch.tensor(train_labels)

# Create a PyTorch dataset
train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'],
                                               train_encodings['attention_mask'],
                                               train_labels)

# Create a data loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

model = model.to(device)

In [13]:
from tqdm.notebook import tqdm
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Set the model to training mode
model.train()

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

losses = []
accuracies = []  # To store accuracy per epoch
num_epochs = 5
# Training loop
for epoch in tqdm(range(num_epochs)):  # Number of training epochs
    running_loss = 0.0
    predicted_labels = []  # To store predicted labels for accuracy calculation
    true_labels = []  # To store true labels for accuracy calculation

    for batch in tqdm(train_loader):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Convert logits to predicted labels
        _, predicted = torch.max(logits, dim=1)
        predicted_labels.extend(predicted.cpu().tolist())
        true_labels.extend(labels.cpu().tolist())

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)

    # Calculate and store accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)
    accuracies.append(accuracy)

    print(f'Epoch {epoch + 1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {accuracy:.4f}')

# Save the model
torch.save(model.state_dict(), 'model.pth')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1256 [00:00<?, ?it/s]

Epoch 1/5 - Loss: 1.0242 - Accuracy: 0.5842


  0%|          | 0/1256 [00:00<?, ?it/s]

...

Epoch 3/5 - Loss: 0.7073 - Accuracy: 0.7381


  0%|          | 0/1256 [00:00<?, ?it/s]

Epoch 4/5 - Loss: 0.6419 - Accuracy: 0.7678


  0%|          | 0/1256 [00:00<?, ?it/s]

Epoch 5/5 - Loss: 0.5806 - Accuracy: 0.7946


In [14]:
#dgfdgdfgdgffdgdfd1212jhkhk

In [15]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

def predict_labels(text):
    train_encodings = tokenizer(text, truncation=True, max_length=128,
        padding='max_length', return_tensors = 'pt')
    input_ids = train_encodings['input_ids'].to(device)
    attention_mask = train_encodings['attention_mask'].to(device)

    # Set the model to evaluation mode
    model.eval()

    # Disable gradient calculation
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)
    

    return predicted_class.item(), probabilities[:,1].item()

In [16]:
predicted_labels = []
predicted_probs = []
for text in tqdm(test_texts):
    predicted_label, prob = predict_labels(text)
    predicted_labels.append(predicted_label)
    predicted_probs.append(prob)

# Calculate accuracy and F1 score
accuracy = accuracy_score(test_labels, predicted_labels)
# f1 = f1_score(test_labels, predicted_labels)
# roc_auc = roc_auc_score(test_labels, predicted_probs)

print('Accuracy:', accuracy)
# print('F1 Score:', f1)
# print('ROC-AUC:', roc_auc)

  0%|          | 0/5022 [00:00<?, ?it/s]

Accuracy: 0.7371565113500598


In [17]:
print('Accuracy:', accuracy)

Accuracy: 0.7371565113500598


In [18]:
from sklearn.metrics import roc_auc_score, classification_report

print('\nThe Classification Report is as follows\n')
print(classification_report(test_labels, predicted_labels, digits = 4))


The Classification Report is as follows

              precision    recall  f1-score   support

           0     0.5240    0.5492    0.5363       417
           1     0.6829    0.6546    0.6684       579
           2     0.8124    0.7453    0.7774      1912
           3     0.0000    0.0000    0.0000        88
           4     0.7333    0.8238    0.7759      2026

    accuracy                         0.7372      5022
   macro avg     0.5505    0.5546    0.5516      5022
weighted avg     0.7274    0.7372    0.7306      5022



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
